In [1]:
import numpy as np

In [2]:
init = [[4,1,2,3],[8,5,6,7],[9,10,11," "]]
goal = [[" ",1,2,3],[4,5,6,7],[8,9,10,11]]
init1 =[[5,1,2,3],[4,6," ",7],[8,9,10,11]]

In [3]:
#step 1
#we have to determine if the goal state is reachable or not.
#what we can do is check the polarity of the initial and goal state if they are
#the same(even-even or odd-odd) then we can reach the goal otherwise we can't

In [4]:
#counts the n number of tiles that are after a tile and are smaller.
def count_smallerTiles(i, init, index):
    current = 0
    current_index = 0
    for item in init:
        for j in item:
            if(j == " " or index != current_index):
                if(index != current_index):
                    current_index = current_index + 1
                continue
            else:
                if (i > j):
                    current = current + 1
    return current

In [5]:
#checking polarity for both states.
def check_polarity(initial_state):
    total = 0
    index = 0
    for item in initial_state:
        for i in item:
            if i == " ":
                free_tile = index 
                index = index + 1
                continue
            else:
                count = count_smallerTiles(i, initial_state, index)
                index = index + 1
                total = total + count
    return total

In [6]:
def check_reachable(initial_state, goal_state):
    check = False
    initial = check_polarity(initial_state)
    print("Inversion count",initial)
    goal = check_polarity(goal_state)
    if(initial % 2 == 0 and goal % 2 == 0):
        print("Goal is reachable as both states have even parity")
        check = True
        return check
    elif(initial % 2 != 0 and goal % 2 != 0):
        print("Goal is reachable as both states have odd parity")
        check = True
        return check
    else:
        print("Goal is not reachable as parity is different for both states")
        return check

In [7]:
#step 2 
#re-arranage the tiles in order to get the goal state. Use IDS algorithm and
#maintain a queue & also a pointer to the empty title.
#so the state space is going to be massive and we can't store it obviously what
#we can do is start with the initial state and keep on expanding until we 
#find the goal state. At each tree level there'll be a new state(each consistent change to the tiles in the state)
#so it's like we are passing the problem(5 comps).

In [8]:
# pip install -U treelib
from treelib import Node, Tree

In [9]:
#returns an array copy of the given obj
def getCopy(state):
    return np.copy(state)

In [10]:
#comparing goal and initial state
def goal_test(initial_state, goal_state):
    if((initial_state == goal_state).all()):
        return initial_state, goal_state

In [11]:
#for moves we do the swap
def swap(state, src, dest):
    i, j = src
    a, b = dest
    swap = state[a][b]
    state[a][b] = state[i][j]
    state[i][j] = swap
    return state

In [12]:
#Step 3 and 4:
#using IDS we will go by each depth level and expand b(branching factor) levels until goal is not 
#found. Using treelib for maintaining the tree DS and the tags are the name for the nodes in the 
#tree. The stack will act as our frontier (will be based on LIFO). Each time we'll
#get the states from the frontier and pop one by one to apply goal test.
#if it's not goal state we'll keeping on increasing the depth and expanding the states until
#the goal state is found or depth limit is reached

In [13]:
def depthLimitedSearch(initial_state, goal_state, maximum_depth):
    tree = Tree() 
    ith = initial_state.shape[1] - 1  #num of rows -> 3
    jth = initial_state.shape[0] - 1  #num of cols -> 4
    tag = 0
    i = 1
    parent_tag = 0
    current_depth = 0
    
    # using stack for the purpose of queue
    stack = [(getCopy(initial_state), 0, 0)] 
    tree.create_node(parent_tag, parent_tag, data = getCopy(initial_state))
    while(len(stack) > 0 and current_depth <= maximum_depth):
        current_state, parent_tag, current_depth = stack.pop()
        result = goal_test(current_state, goal_state)
        if(result):
            print('Goal path:')
            for state in stack:
                print(state[1], end=" ")
            print('\n')
            print('Tree:')
            tree.show()            
            return current_depth        
        if(current_depth == maximum_depth):
            continue
        pointer = np.where(current_state == " ")
        new_tile = pointer[0][0], pointer[1][0]
        row, col = new_tile
        temp = getCopy(current_state)
        
         # up move
        if row - 1 >= 0:
            temp = swap(getCopy(current_state), new_tile, (row - 1, col))
            tree.create_node(tag + i, tag + i, data = getCopy(temp), parent = parent_tag)
            stack.append((getCopy(temp), tag + i, current_depth + 1))
            i = i + 1
            
         # down move
        if row + 1 < ith:
            temp = swap(getCopy(current_state), new_tile, (row + 1, col))
            tree.create_node(tag + i, tag + i, data = getCopy(temp), parent = parent_tag)
            stack.append((getCopy(temp), tag + i, current_depth + 1))
            i = i + 1
            
        # right move
        if col + 1 < jth:
            temp = swap(getCopy(current_state), new_tile, (row, col + 1))
            tree.create_node(tag + i, tag + i, data = getCopy(temp), parent = parent_tag)
            stack.append((getCopy(temp), tag + i, current_depth + 1))
            i = i + 1

        # left move
        if col - 1 >= 0:
            temp = swap(getCopy(current_state), new_tile, (row, col - 1))
            tree.create_node(tag + i, tag + i, data = getCopy(temp), parent = parent_tag)
            stack.append((getCopy(temp), tag + i, current_depth + 1))
            i = i + 1

        print('Queue:')
        for state in stack:
            print(state[1], end=" ")
        print('\n')
        
    print('Expansion of states in tree representation:')  
    tree.show()
    print("")
    return False

In [14]:
def iterativeDeepening(initial_state, goal_state, maximum_depth):
    depth = 0
    solution = False
    while(not solution and depth <= maximum_depth):
        solution = depthLimitedSearch(initial_state, goal_state, depth)
        depth = depth + 1
    return solution

In [15]:
def puzzle(initial_state, goal_state):
    reachable = check_reachable(initial_state, goal_state)
    if(reachable):
        maximum_depth = 10
        initial_state = np.array(initial_state)
        goal_state = np.array(goal_state)
        res = iterativeDeepening(initial_state, goal_state, maximum_depth)
        if res:
            print("Goal state found at depth level : ", res)
        else:
            print("Maximum depth reached and goal state not found")

In [16]:
puzzle(init, goal) 
# puzzle(init1, goal)

Inversion count 6
Goal is reachable as both states have even parity
Expansion of states in tree representation:
0


Queue:
1 2 

Expansion of states in tree representation:
0
├── 1
└── 2


Queue:
1 2 

Queue:
1 3 4 

Queue:
5 6 7 

Expansion of states in tree representation:
0
├── 1
│   ├── 5
│   ├── 6
│   └── 7
└── 2
    ├── 3
    └── 4


Queue:
1 2 

Queue:
1 3 4 

Queue:
1 3 5 6 

Queue:
1 7 8 9 

Queue:
10 11 12 

Queue:
10 11 13 14 15 

Queue:
10 16 17 

Queue:
18 19 

Expansion of states in tree representation:
0
├── 1
│   ├── 10
│   │   ├── 18
│   │   └── 19
│   ├── 11
│   │   ├── 16
│   │   └── 17
│   └── 12
│       ├── 13
│       ├── 14
│       └── 15
└── 2
    ├── 3
    │   ├── 7
    │   ├── 8
    │   └── 9
    └── 4
        ├── 5
        └── 6


Queue:
1 2 

Queue:
1 3 4 

Queue:
1 3 5 6 

Queue:
1 3 5 7 8 

Queue:
1 3 9 10 11 

Queue:
1 12 13 14 

Queue:
1 12 13 15 16 17 

Queue:
1 12 18 19 

Queue:
1 20 21 

Queue:
22 23 24 

Queue:
22 23 25 26 27 

Queue:
22 23 25 26 28 2